In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


# Collecting Dataset

In [4]:
DATASET_DIR = '../datasets/UCF101/'

In [5]:
df = pd.read_csv(DATASET_DIR+ 'train.csv')
df['clip_path'] = df['clip_path'].apply(lambda x: DATASET_DIR + x) 
df.head()

,clip_name,clip_path,label
0,v_Swing_g05_c02,../datasets/UCF101//train/Swing/v_Swing_g05_c0...,Swing
1,v_Swing_g21_c03,../datasets/UCF101//train/Swing/v_Swing_g21_c0...,Swing
2,v_Swing_g07_c01,../datasets/UCF101//train/Swing/v_Swing_g07_c0...,Swing
3,v_Swing_g24_c04,../datasets/UCF101//train/Swing/v_Swing_g24_c0...,Swing
4,v_Swing_g20_c03,../datasets/UCF101//train/Swing/v_Swing_g20_c0...,Swing


In [6]:
df['label'].unique()

array(['Swing', 'SkyDiving', 'BreastStroke', 'TableTennisShot', 'FloorGymnastics', 'Rowing', 'SoccerJuggling', 'BenchPress', 'TennisSwing', 'FrisbeeCatch', 'Kayaking', 'ApplyLipstick', 'RopeClimbing', 'JumpingJack', 'BlowingCandles', 'BabyCrawling', 'HammerThrow', 'BoxingPunchingBag', 'Skiing', 'UnevenBars',
       'PullUps', 'RockClimbingIndoor', 'HeadMassage', 'HulaHoop', 'Rafting', 'ShavingBeard', 'LongJump', 'FieldHockeyPenalty', 'VolleyballSpiking', 'Punch', 'Archery', 'PlayingDhol', 'CleanAndJerk', 'PlayingDaf', 'GolfSwing', 'PlayingSitar', 'IceDancing', 'SkateBoarding', 'BodyWeightSquats', 'HandstandPushups',
       'BalanceBeam', 'WallPushups', 'HorseRiding', 'Bowling', 'JumpRope', 'CuttingInKitchen', 'CliffDiving', 'PlayingGuitar', 'FrontCrawl', 'HandstandWalking', 'Fencing', 'Basketball', 'Typing', 'TaiChi', 'PlayingCello', 'Lunges', 'Shotput', 'Nunchucks', 'ApplyEyeMakeup', 'BaseballPitch', 'Mixing', 'PlayingTabla',
       'BandMarching', 'MoppingFloor', 'PushUps', 'Military

In [7]:
punch_df = df[df['label']=='Punch'].reset_index(drop=True)
box_p_bag_df = df[df['label']=='BoxingPunchingBag'].reset_index(drop=True)
walking_df = df[df['label']=='WalkingWithDog'].reset_index(drop=True)

In [8]:
train_dir =  list(walking_df['clip_path']) + list(box_p_bag_df['clip_path']) + list(punch_df['clip_path'])
train_label = list( walking_df['label']) + list(box_p_bag_df['label']) + list(punch_df['label'])


len(train_dir)

334

# Infrence on Dataset

In [9]:
from ultralytics import YOLO
import cv2

In [10]:
model = YOLO('models/yolov8x-pose-p6.pt') 

In [14]:
vp = VideoProcessor(model)

keypoints_df = pd.DataFrame()
for train_df in [walking_df, box_p_bag_df, punch_df]:  #
    label = train_df['label'][0]
    print(f"Training: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,1,20,34)
    class_keypoints_df = pd.concat([pd.DataFrame(batch.tolist()), pd.DataFrame([label]*len(batch))], axis=1)
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])

    

cv2.destroyAllWindows()

Training: WalkingWithDog


Processing Videos:  98%|█████████▊| 90/92 [13:40<00:18,  9.12s/it]


Training: BoxingPunchingBag


Processing Videos:  99%|█████████▉| 121/122 [15:58<00:07,  7.92s/it]


Training: Punch


Processing Videos:  95%|█████████▌| 114/120 [16:11<00:51,  8.52s/it]


# convert to datset

In [15]:
# keypoints_df = pd.concat([pd.DataFrame(t.tolist()),  pd.DataFrame(df['clip_path']), pd.DataFrame(df['label'])], axis=1)

In [16]:
keypoints_df

,0,0
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46...",WalkingWithDog
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",WalkingWithDog
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",WalkingWithDog
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",WalkingWithDog
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",WalkingWithDog
...,...,...
1620,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18...",Punch
1621,"[[0.6189828395843506, 0.19277691841125488, 0.6...",Punch
1622,"[[0.7276927471160889, 0.16618148485819498, 0.7...",Punch
1623,"[[0.6366876125335693, 0.2523201942443848, 0.64...",Punch


In [17]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)